In [1]:
import pandas as pd
import numpy as np
import urllib2
import webbrowser
import os
import bs4,  re
from bs4 import BeautifulSoup
import requests
import math
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import csv
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

import plotly.graph_objs as go

In [2]:
df = pd.read_csv('D:\Python_Database\Myeloma\CART Results.csv')
df.columns

Index([u'Rank', u'NCT Number', u'Title', u'Acronym', u'Status',
       u'Study Results', u'Conditions', u'Interventions', u'Outcome Measures',
       u'Sponsor/Collaborators', u'Gender', u'Age', u'Phases', u'Enrollment',
       u'Funded Bys', u'Study Type', u'Study Designs', u'Other IDs',
       u'Start Date', u'Primary Completion Date', u'Completion Date',
       u'First Posted', u'Results First Posted', u'Last Update Posted',
       u'Locations', u'Study Documents', u'URL'],
      dtype='object')

In [3]:
abstract = pd.read_csv('D://Python_Database//Myeloma//Ash2018//abstract.csv')
title = pd.read_csv('D://Python_Database//Myeloma//Ash2018//titles.csv')
abstract = abstract['Paper Abstract'].tolist()
title = title['Paper Title'].tolist()


In [4]:
df = df[['NCT Number', 'Title', 'Interventions']]
df

,NCT Number,Title,Interventions
0,NCT03549442,Up-front CART-BCMA With or Without huCART19 in...,Combination Product: BCMA CART + huCART19|Comb...
1,NCT03464916,Study to Evaluate the Safety and Efficacy of A...,Biological: CAR2 Anti-CD38 A2 CAR-T Cells
2,NCT02794246,CART-19 Post-ASCT for Multiple Myeloma,Biological: CART-19 cells
3,NCT02546167,CART-BCMA Cells for Multiple Myeloma,Biological: CART-BCMA
4,NCT03502577,BCMA-Specific CAR T-Cells Combined With a Gamm...,Biological: BCMA-specific CAR-expressing T Lym...
5,NCT03070327,BCMA Targeted CAR T Cells With or Without Lena...,Biological: EGFRt/BCMA-41BBz CAR T cell|Drug: ...
6,NCT02135406,CART-19 for Multiple Myeloma,Biological: CART-19 T cells
7,NCT03710421,CS1-CAR T Therapy Following Chemotherapy in Tr...,Procedure: CS1-CAR T Therapy|Drug: Cyclophosph...
8,NCT03338972,Immunotherapy With BCMA CAR-T Cells in Treatin...,Biological: Autologous Anti-BCMA-CAR-expressin...
9,NCT03548207,"A Study of JNJ-68284528, a Chimeric Antigen Re...",Biological: JNJ-68284528


In [5]:
NCT_No = df['NCT Number'].tolist()
NCT_No

['NCT03549442',
 'NCT03464916',
 'NCT02794246',
 'NCT02546167',
 'NCT03502577',
 'NCT03070327',
 'NCT02135406',
 'NCT03710421',
 'NCT03338972',
 'NCT03548207',
 'NCT03672318',
 'NCT03448978',
 'NCT03602612',
 'NCT03288493',
 'NCT02215967',
 'NCT03651128',
 'NCT03318861',
 'NCT02203825',
 'NCT03274219',
 'NCT03361748',
 'NCT03430011',
 'NCT02658929',
 'NCT02529813']

In [6]:
numbers = []
trial = []
pos = []
for i in NCT_No:
    for j in range(len(abstract)):
        if i in abstract[j]:
            numbers.append(j)
            trial.append(i)
            pos.append(NCT_No.index(i))
        else:
            continue

In [7]:
numbers, trial, pos

([1069, 2219, 2716, 668, 3359, 1936],
 ['NCT02546167',
  'NCT03548207',
  'NCT03288493',
  'NCT03274219',
  'NCT03430011',
  'NCT02658929'],
 [3, 9, 13, 18, 20, 21])

In [90]:
abstract[1069]

'[<div class="abstract">\r\n<div><strong>Introduction:</strong> The optimal clinical setting and cell product characteristics for chimeric antigen receptor (CAR) T cell therapy in multiple myeloma (MM) are uncertain. In CLL patients treated with anti-CD19 CAR T cells (CART19), prevalence of an early memory (early-mem) T cell phenotype (CD27+ CD45RO- CD8+) at time of leukapheresis was predictive of clinical response independently of other patient- or disease-specific factors and was associated with enhanced capacity for in vitro T cell expansion and CD19-responsive activation (Fraietta et al. Nat Med 2018). T cell fitness is therefore a major determinant of response to CAR T cell therapy. In an accompanying abstract (Cohen et al.), we report that higher percentage of early-mem T cells and CD4/CD8 ratio within the leukapheresis product are associated with favorable clinical response to anti-BCMA CAR T cells (CART-BCMA) in relapsed/refractory MM. Here, we compare leukapheresis samples fro

In [8]:
title[1069]

'<title>Paper: Clinical Predictors of T Cell Fitness for CAR T Cell Manufacturing and Efficacy in Multiple Myeloma</title>'

In [9]:
papers = []

for i in numbers:
    papers.append(title[i])


In [11]:
paper = []
for i in range(len(papers)):
    paper.append(papers[i][14:-8])
    
paper

['Clinical Predictors of T Cell Fitness for CAR T Cell Manufacturing and Efficacy in Multiple Myeloma',
 'Updated Analysis of a Phase 1, Open-Label Study of LCAR-B38M, a Chimeric Antigen Receptor T Cell Therapy Directed Against B-Cell Maturation Antigen, in Patients with Relapsed/Refractory Multiple Myeloma',
 'Efficacy and Safety of P-Bcma-101 CAR-T Cells in Patients with Relapsed/Refractory (r/r) Multiple Myeloma (MM)',
 'Initial Results from a Phase 1 Clinical Study of bb21217, a Next-Generation Anti Bcma CAR T Therapy',
 'JCARH125, Anti-BCMA CAR T-cell Therapy for Relapsed/Refractory Multiple Myeloma: Initial Proof of Concept Results from a Phase 1/2 Multicenter Study (EVOLVE)',
 'ET190L1-Artemis<sup>TM</sup> T Cell Therapy Results in Durable Disease Remissions with No Cytokine Release Syndrome or Neurotoxicity in Patients with Relapsed and Refractory B-Cell Lymphoma']

In [12]:
trial_title = []
for i in pos:
    trial_title.append(df.iloc[i]['Title'])
    
trial_title

['CART-BCMA Cells for Multiple Myeloma',
 'A Study of JNJ-68284528, a Chimeric Antigen Receptor T Cell (CAR-T) Therapy Directed Against B-Cell Maturation Antigen (BCMA) in Participants With Relapsed or Refractory Multiple Myeloma',
 'P-BCMA-101 Tscm CAR-T Cells in the Treatment of Patients With Multiple Myeloma (MM)',
 'Study of bb21217 in Multiple Myeloma',
 'Study Evaluating the Safety and Efficacy of JCARH125 in Subjects With Relapsed and/or Refractory Multiple Myeloma',
 'Study of bb2121 in Multiple Myeloma']

In [13]:
product = pd.DataFrame(list(map(list, zip(trial,trial_title,paper))))
product.columns = ['Trial Number', 'Trial Title', 'ASH2018 Paper Title']
pd.options.display.max_colwidth = 0

In [14]:
product

,Trial Number,Trial Title,ASH2018 Paper Title
0,NCT02546167,CART-BCMA Cells for Multiple Myeloma,Clinical Predictors of T Cell Fitness for CAR T Cell Manufacturing and Efficacy in Multiple Myeloma
1,NCT03548207,"A Study of JNJ-68284528, a Chimeric Antigen Receptor T Cell (CAR-T) Therapy Directed Against B-Cell Maturation Antigen (BCMA) in Participants With Relapsed or Refractory Multiple Myeloma","Updated Analysis of a Phase 1, Open-Label Study of LCAR-B38M, a Chimeric Antigen Receptor T Cell Therapy Directed Against B-Cell Maturation Antigen, in Patients with Relapsed/Refractory Multiple Myeloma"
2,NCT03288493,P-BCMA-101 Tscm CAR-T Cells in the Treatment of Patients With Multiple Myeloma (MM),Efficacy and Safety of P-Bcma-101 CAR-T Cells in Patients with Relapsed/Refractory (r/r) Multiple Myeloma (MM)
3,NCT03274219,Study of bb21217 in Multiple Myeloma,"Initial Results from a Phase 1 Clinical Study of bb21217, a Next-Generation Anti Bcma CAR T Therapy"
4,NCT03430011,Study Evaluating the Safety and Efficacy of JCARH125 in Subjects With Relapsed and/or Refractory Multiple Myeloma,"JCARH125, Anti-BCMA CAR T-cell Therapy for Relapsed/Refractory Multiple Myeloma: Initial Proof of Concept Results from a Phase 1/2 Multicenter Study (EVOLVE)"
5,NCT02658929,Study of bb2121 in Multiple Myeloma,ET190L1-Artemis<sup>TM</sup> T Cell Therapy Results in Durable Disease Remissions with No Cytokine Release Syndrome or Neurotoxicity in Patients with Relapsed and Refractory B-Cell Lymphoma


In [15]:
def ASH2018_papers(word1, word2, word3):
    num = []
    for i in range(len(abstract)):
        if word1 in abstract[i] and word2 in abstract[i] and word3 in abstract[i]:
            num.append(i)
        else:
            continue
            
    filtered_lst = set(num).difference(numbers)
    
    papers = []
    for i in filtered_lst:
        papers.append(title[i])
    
    ASH_paper_title = []
    for i in range(len(papers)):
        ASH_paper_title.append(papers[i][14:-8])
        
    ASH_paper_title = pd.DataFrame(ASH_paper_title)
    ASH_paper_title.columns = ['ASH 2018 Titles']
        
    return ASH_paper_title, num
    

In [16]:
BCMA = ASH2018_papers('myeloma', 'BCMA', 'CAR')
BCMA_lst = zip(BCMA[1], ['BCMA']*len(BCMA[1]))
BCMA_lst

[(308, 'BCMA'),
 (668, 'BCMA'),
 (1065, 'BCMA'),
 (1069, 'BCMA'),
 (1359, 'BCMA'),
 (1394, 'BCMA'),
 (1408, 'BCMA'),
 (1490, 'BCMA'),
 (1693, 'BCMA'),
 (1756, 'BCMA'),
 (2045, 'BCMA'),
 (2219, 'BCMA'),
 (2716, 'BCMA'),
 (2812, 'BCMA'),
 (2883, 'BCMA'),
 (3166, 'BCMA'),
 (3359, 'BCMA'),
 (3389, 'BCMA'),
 (3712, 'BCMA'),
 (4229, 'BCMA'),
 (4281, 'BCMA'),
 (4363, 'BCMA'),
 (4382, 'BCMA')]

In [17]:
title[308]

'<title>Paper: A Phase 1 Study of Two Investigational Agents, ACTR087, an Autologous T Cell Product Expressing an Antibody-Coupled T Cell Receptor, in Combination with SEA-BCMA, a Novel Non-Fucosylated Monoclonal Antibody, in Subjects with Relapsed or Refractory Multiple Myeloma</title>'

In [18]:
BCMA[0]

,ASH 2018 Titles
0,"REGN5458, a Bispecific BCMAxCD3 T Cell Engaging Antibody, Demonstrates Robust In Vitro and In Vivo Anti-Tumor Efficacy in Multiple Myeloma Models, Comparable to That of BCMA CAR T Cells"
1,"CAR T Cell Therapy Targeting G Protein-Coupled Receptor Class C Group 5 Member D (GPRC5D), a Novel Target for the Immunotherapy of Multiple Myeloma"
2,Low Dose of Human scFv-Derived BCMA-Targeted CAR-T Cells Achieved Fast Response and High Complete Remission in Patients with Relapsed/Refractory Multiple Myeloma
3,"Chemically Controlled, Immunosuppression-Resistant, Anti-Bcma CAR-T Cells for Treatment of Antibody-Mediated Autoimmunity"
4,Engineering an Optimized Trimeric APRIL-Based CAR to Broaden Targetability of Multiple Myeloma
5,<em> Identifying Factors in Multiple Myeloma Controlling Response to B-Cell Maturation Antigen (BCMA)-Targeted Immunotherapy Using CRISPR-Based Functional Genomics</em>
6,"Preclinical Development of CTX120, an Allogeneic CAR-T Cell Targeting Bcma"
7,"ALLO-715, an Allogeneic Bcma CAR T Therapy Possessing an Off-Switch for the Treatment of Multiple Myeloma"
8,"Clinical Responses and Pharmacokinetics of MCARH171, a Human-Derived Bcma Targeted CAR T Cell Therapy in Relapsed/Refractory Multiple Myeloma: Final Results of a Phase I Clinical Trial"
9,Durable Remission Achieved from Bcma-Directed CAR-T Therapy Against Relapsed or Refractory Multiple Myeloma


In [102]:
CD19 = ASH2018_papers('myeloma', 'CD19', 'CAR')

CD19 = ASH2018_papers('myeloma', 'CD19', 'CAR')
CD19_lst = zip(CD19[1], ['CD19']*len(CD19[1]))
CD19_lst
# CD19[0]

[(1065, 'CD19'),
 (1069, 'CD19'),
 (1336, 'CD19'),
 (1394, 'CD19'),
 (2985, 'CD19'),
 (3166, 'CD19'),
 (3389, 'CD19')]

In [103]:
CD138 = ASH2018_papers('myeloma', 'CD138', 'CAR')

CD138 = ASH2018_papers('myeloma', 'CD138', 'CAR')
CD138_lst = zip(CD138[1], ['CD138']*len(CD138[1]))
CD138_lst

# CD138[0]

[(1509, 'CD138'), (2985, 'CD138'), (3166, 'CD138'), (3712, 'CD138')]

In [104]:
CD38 = ASH2018_papers('myeloma', 'CD38', 'CAR')

CD38 = ASH2018_papers('myeloma', 'CD38', 'CAR')
CD38_lst = zip(CD38[1], ['CD38']*len(CD38[1]))
CD38_lst

# CD38[0]

[(1620, 'CD38'),
 (1693, 'CD38'),
 (1964, 'CD38'),
 (2459, 'CD38'),
 (2883, 'CD38'),
 (2985, 'CD38'),
 (3359, 'CD38'),
 (3712, 'CD38')]

In [105]:
NK = ASH2018_papers('myeloma', 'NK', 'CAR')

NK = ASH2018_papers('myeloma', 'NK', 'CAR')
NK_lst = zip(NK[1], ['NK38']*len(NK[1]))
NK_lst

# NK[0]

[(610, 'NK38'), (1848, 'NK38')]

In [106]:
CART_abstract_title_no = BCMA_lst + CD19_lst + CD138_lst + NK_lst + CD38_lst

In [107]:
num = []
for i in range(len(CART_abstract_title_no)):
    num.append(CART_abstract_title_no[i][0])
    
num   

[308,
 668,
 1065,
 1069,
 1359,
 1394,
 1408,
 1490,
 1693,
 1756,
 2045,
 2219,
 2716,
 2812,
 2883,
 3166,
 3359,
 3389,
 3712,
 4229,
 4281,
 4363,
 4382,
 1065,
 1069,
 1336,
 1394,
 2985,
 3166,
 3389,
 1509,
 2985,
 3166,
 3712,
 610,
 1848,
 1620,
 1693,
 1964,
 2459,
 2883,
 2985,
 3359,
 3712]

In [108]:
CAR_Titles, CAR_Abstract = [], []
for i in range(len(num)):
    CAR_Titles.append(title1[num[i]])
    CAR_Abstract.append(abstract[i])


In [252]:

patients = []
for i in range(len(CAR_Abstract)):
    lst = CAR_Abstract[i].split()
    temp = []
    for j in range(len(lst)):
        if lst[j] == 'patients' and lst[j-1].isdigit():
            temp.append(lst[j-1])

    patients.append(temp)
   

        
#         patients = [i for i in patients if i.isdigit()]         
# patients = [int(i) for i in patients]
# patients = np.amax(np.array([patients])).astype(int)
# print 'number of patients = ' + str(patients)

In [253]:
filt1=[]
for i in range(len(patients)):
    if patients[i] == []:
        filt1.append(i)
        
        
len(filt1), patients

(23,
 [['93', '67', '48', '30', '18'],
  ['143'],
  ['71', '13', '6', '4'],
  ['140', '93', '47'],
  [],
  [],
  ['60'],
  [],
  ['172', '240'],
  ['367'],
  ['10', '6', '37', '3'],
  ['108', '61'],
  [],
  ['57'],
  [],
  [],
  [],
  [],
  ['25', '50'],
  ['11439'],
  ['24'],
  ['8', '2'],
  ['108'],
  ['309'],
  [],
  ['79', '31', '168', '317', '44', '15', '41', '15', '18', '32'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['142', '17', '3'],
  [],
  ['19', '10'],
  [],
  [],
  [],
  ['401', '60'],
  [],
  ['2']])

In [255]:
temp = []
for j in range(len(patients)):
    if patients[j] != []:
        temp.append(patients[j])


temp



[['93', '67', '48', '30', '18'],
 ['143'],
 ['71', '13', '6', '4'],
 ['140', '93', '47'],
 ['60'],
 ['172', '240'],
 ['367'],
 ['10', '6', '37', '3'],
 ['108', '61'],
 ['57'],
 ['25', '50'],
 ['11439'],
 ['24'],
 ['8', '2'],
 ['108'],
 ['309'],
 ['79', '31', '168', '317', '44', '15', '41', '15', '18', '32'],
 ['142', '17', '3'],
 ['19', '10'],
 ['401', '60'],
 ['2']]

In [157]:
CAR_Titles[0]

'<title>Paper: Initial Results from a Phase 1 Clinical Study of bb21217, a Next-Generation Anti Bcma CAR T Therapy</title>'

In [164]:
lst = CAR_Abstract[0].split()

In [165]:
patients = []
for i in range(len(lst)):
    if lst[i] == 'patients' and lst[i-1].isdigit():
        patients.append(lst[i-1])
    else:
        patients.append('Not Applicable')
        
patients = [i for i in patients if i.isdigit()]        
patients = [int(i) for i in patients]
patients = np.amax(np.array([patients])).astype(int)
print 'number of patients = ' + str(patients)

number of patients = 143


In [256]:
CAR_Abstract[0]

'[<div class="abstract">\r\n<div>Several prognostic models have been developed to predict survival outcomes and response in patients with myelofibrosis (MF). MIPSS70 prognostic system, developed by incorporation of all the key clinical characteristics, cytogenetics, and mutational factors into one system, has recently been revised to MIPSS70+ v2.0 with refinements in degrees of anemia, cytogenetics, and HMR. While allogeneic hematopoietic cell transplantation (alloHCT) is the only curative treatment for patients with MF, limited data exists on the impact of molecular markers on transplant outcomes. Here, we evaluated the transplant outcome in MF patients who uniformly received fludarabine/melphalan (FluMel) conditioning at City of Hope and assessed the impact of cytogenetics, somatic mutations on transplant outcomes based on a 72 gene next-generation sequencing (NGS) panel and MIPSS 70+ v2.0.\r\n<p>A total of 110 consecutive MF patients (primary: n=58, secondary: n=52) without prior ac

In [258]:
CAR_Abstract[19]

'[<div class="abstract">\r\n<div><strong>Background</strong>: Acute myeloid leukemia (AML) secondary to haematological neoplasia or other malignant diseases, defined as secondary AML (sAML), is thought to have an inferior prognosis as compared with de novo AML. However, when corrected for other risk factors such as age and cytogenetic risk, differences are less clear. In particular, it is uncertain whether outcome after allogeneic stem cell transplantation (alloSCT) is different. Thus, the Acute Leukemia Working Party (ALWP) of the European Society for Blood and Marrow Transplantation (EBMT) performed a registry-based analysis to compare results after alloSCT for sAML versus de novo AML.\r\n<p><strong>Methods:</strong> Inclusion criteria were age &gt;= 18y and alloSCT for de novo or sAML from a matched related, matched unrelated or T-cell replete haploidentical donor between 2000 and 2016. Patients without information on cytogenetics were excluded. 11439 patients with de novo and 1325 

In [259]:
CAR_Titles[19]

'<title>Paper: Engineering an Optimized Trimeric APRIL-Based CAR to Broaden Targetability of Multiple Myeloma</title>'